link to HF tutorial:
https://huggingface.co/blog/fine-tune-whisper

In [97]:
from datasets import load_dataset, DatasetDict, Audio

from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union

import evaluate
metric = evaluate.load("wer")

In [98]:
dataset = DatasetDict()

dataset['train'] = load_dataset('rodoggx/ATCO2-ASR-1h', split='train')
dataset['test'] = load_dataset('rodoggx/ATCO2-ASR-1h', split='test')

In [99]:
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="english", task="transcribe")
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Hindi", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [100]:
input_str = dataset['train'][0]["text Str"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print("Input text:", input_str)
print("Labels:", labels)
print("Decoded with special tokens:", decoded_with_special)
print("Decoded without special tokens:", decoded_str)
print('is equal:', input_str == decoded_str)

Input text: tower good day New Zealand Two Thirty One seven charlie three four left
Labels: [50258, 50259, 50359, 50363, 83, 968, 665, 786, 1873, 13883, 4453, 41490, 1485, 3407, 1290, 6302, 1045, 1451, 1411, 50257]
Decoded with special tokens: <|startoftranscript|><|en|><|transcribe|><|notimestamps|>tower good day New Zealand Two Thirty One seven charlie three four left<|endoftext|>
Decoded without special tokens: tower good day New Zealand Two Thirty One seven charlie three four left
is equal: True


In [101]:
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [102]:
def prepare_datset(batch):
    audio = batch["audio"]
    batch["input_values"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    batch['labels'] = tokenizer(batch['text Str']).input_ids
    return batch

dataset = dataset.map(prepare_datset, remove_columns=dataset.column_names['train'], batch_size=8, num_proc=4)

In [103]:
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

model.generation_config.language = "english"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

In [104]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [105]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [106]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    labels = pred.label_ids
    labels[labels == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(labels, skip_special_tokens=True)
    wer = metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

In [107]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-en",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=5000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    include_inputs_for_metrics=True,
    push_to_hub=False,
)

/home/rodoggx/Documents/iResearch-2024/venv/lib/python3.11/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [108]:
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)
trainer

max_steps is given, it will override any value given in num_train_epochs


In [109]:
trainer.train()

  0%|          | 0/5000 [00:00<?, ?it/s]

KeyError: 'input_features'